In [1]:
import tensorflow as tf 
from tensorflow.keras import datasets,layers,models
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
img_size=224
batch_size=32

In [30]:
data=tf.keras.preprocessing.image_dataset_from_directory(
    "resized_dataset",
    shuffle=True,
    image_size=(img_size,img_size),
    batch_size=batch_size
    
)

Found 869 files belonging to 6 classes.


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#spliting

data=ImageDataGenerator(
    rescale=1./255,          # Normalize the images to the range [0, 1]
    rotation_range=20,       # Random rotations
    width_shift_range=0.2,   # Random width shifts
    height_shift_range=0.2,  # Random height shifts
    shear_range=0.2,         # Shearing transformations
    zoom_range=0.2,          # Zoom transformations
    horizontal_flip=True,    # Random horizontal flips
    fill_mode='nearest', 
    validation_split=0.2
)

In [32]:
base_dire='resized_dataset'

In [33]:
#train model

train_gen=data.flow_from_directory(
    base_dire,
    target_size=(img_size,img_size),
    batch_size=batch_size, 
    subset='training', 
    class_mode='categorical'
)

Found 698 images belonging to 6 classes.


In [34]:
#validation 

train_val=data.flow_from_directory(
    base_dire, 
    target_size=(img_size,img_size), 
    batch_size=batch_size, 
    subset='validation', 
    class_mode='categorical'
)

Found 171 images belonging to 6 classes.


In [35]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam

# Define the model using ResNet50
pretrained_model = ResNet50(include_top=False, 
                            input_shape=(224, 224, 3),  # 224x224 for ResNet50
                            pooling='avg',  # Global Average Pooling
                            weights='imagenet')  # Pretrained weights

# Freeze all layers of the ResNet50 model to prevent fine-tuning
for layer in pretrained_model.layers:
    layer.trainable = False

# Create the final model
model = models.Sequential()
model.add(pretrained_model)  # Add the ResNet50 model
model.add(layers.Dense(512, activation='relu'))  # Fully connected layer
model.add(layers.Dense(6, activation='softmax'))  # Output layer (5 classes)




In [36]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,639,878 (93.99 MB)

 Trainable params: 1,052,166 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [37]:
#compile the model

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [38]:
#training model

history=model.fit(
    train_gen,
    steps_per_epoch= train_gen.samples // batch_size, 
    epochs=30, 
    validation_data= train_val, 
    validation_steps= train_val.samples // batch_size    
)

C:\Users\wicky\tf-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 291s 13s/step - accuracy: 0.2159 - loss: 2.0313 - val_accuracy: 0.2750 - val_loss: 1.7276
Epoch 2/30
 1/21 ━━━━━━━━━━━━━━━━━━━━ 3:10 10s/step - accuracy: 0.1562 - loss: 1.9375

C:\Users\wicky\tf-env\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


21/21 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.1562 - loss: 1.9375 - val_accuracy: 0.2313 - val_loss: 1.6922
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 255s 12s/step - accuracy: 0.2977 - loss: 1.7279 - val_accuracy: 0.3250 - val_loss: 1.6147
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.3438 - loss: 1.5288 - val_accuracy: 0.3438 - val_loss: 1.6144
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 258s 12s/step - accuracy: 0.3186 - loss: 1.6516 - val_accuracy: 0.3250 - val_loss: 1.6066
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.3750 - loss: 1.6559 - val_accuracy: 0.3688 - val_loss: 1.6050
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 255s 12s/step - accuracy: 0.3002 - loss: 1.6320 - val_accuracy: 0.3063 - val_loss: 1.6043
Epoch 8/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.3750 - loss: 1.5876 - val_accuracy: 0.3688 - val_loss: 1.5587
Epoch 9/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 260s 12s/step - accuracy: 0.3801 - loss: 1.5360 - val_accuracy: 0.3438 - val_loss: 1.

In [39]:
#evaluting
print("evaluting model")
val_loss,val_accuracy= model.evaluate(train_val,steps=train_val.samples // batch_size)
print(f"validation accuracy: {val_accuracy * 100:.2f} ")

evaluting model
5/5 ━━━━━━━━━━━━━━━━━━━━ 57s 12s/step - accuracy: 0.3835 - loss: 1.4729
validation accuracy: 41.87 


In [40]:
model.save('solar_yy.h5')
